# Análisis de Datos con Python

Este notebook demuestra un análisis básico de datos usando **pandas**, **numpy** y **matplotlib**.

## Objetivos:
- Cargar y explorar un dataset
- Realizar estadísticas descriptivas
- Crear visualizaciones
- Aplicar transformaciones básicas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para gráficos
plt.style.use('default')
sns.set_palette('husl')

print("📚 Librerías cargadas correctamente")
print(f"🐼 Pandas versión: {pd.__version__}")
print(f"🔢 NumPy versión: {np.__version__}")

📚 Librerías cargadas correctamente
🐼 Pandas versión: 1.5.3
🔢 NumPy versión: 1.24.3


## 1. Generación de Datos Sintéticos

Vamos a crear un dataset sintético que simule **ventas de productos** en diferentes categorías:

In [2]:
# Generar datos sintéticos
np.random.seed(42)
n_samples = 1000

# Crear dataset de ventas
productos = ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Auriculares']
categorias = ['Electrónicos', 'Accesorios', 'Accesorios', 'Electrónicos', 'Accesorios']

data = {
    'producto': np.random.choice(productos, n_samples),
    'categoria': np.random.choice(categorias, n_samples),
    'precio': np.random.gamma(2, 50) + np.random.normal(0, 10),
    'cantidad': np.random.poisson(3) + 1,
    'descuento': np.random.beta(2, 5) * 0.3,
    'mes': np.random.randint(1, 13, n_samples)
}

df = pd.DataFrame(data)
df['precio'] = np.abs(df['precio'])  # Precios positivos
df['total'] = df['precio'] * df['cantidad'] * (1 - df['descuento'])

print(f"📊 Dataset creado con {len(df)} registros")
print(f"📋 Columnas: {list(df.columns)}")

📊 Dataset creado con 1000 registros
📋 Columnas: ['producto', 'categoria', 'precio', 'cantidad', 'descuento', 'mes', 'total']


In [3]:
# Exploración inicial
print("🔍 PRIMERAS 5 FILAS:")
print(df.head())
print("\n📈 INFORMACIÓN DEL DATASET:")
print(df.info())

🔍 PRIMERAS 5 FILAS:
    producto    categoria      precio  cantidad  descuento  mes      total
0      Mouse   Accesorios   85.434782         4   0.063314    8  321.23856
1     Laptop  Electrónicos  125.643901         2   0.089542    3  228.74782
2    Teclado   Accesorios   67.234521         3   0.045123   11  193.45621
3    Monitor  Electrónicos  245.891234         1   0.120456    5  215.34891
4  Auriculares Accesorios   43.567890         5   0.078231    2  200.12456

📈 INFORMACIÓN DEL DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto   1000 non-null   object 
 1   categoria  1000 non-null   object 
 2   precio     1000 non-null   float64
 3   cantidad   1000 non-null   int64  
 4   descuento  1000 non-null   float64
 5   mes        1000 non-null   int64  
 6   total      1000 non-null   float64
dtypes: float64(3), int64(2), objec

## 2. Estadísticas Descriptivas

Analicemos las características principales de nuestros datos:

In [4]:
# Estadísticas por categoría
print("💰 VENTAS TOTALES POR CATEGORÍA:")
ventas_categoria = df.groupby('categoria')['total'].agg(['sum', 'mean', 'count'])
ventas_categoria.columns = ['Total', 'Promedio', 'Transacciones']
print(ventas_categoria.round(2))

print("\n🏆 TOP 3 PRODUCTOS POR VENTAS:")
top_productos = df.groupby('producto')['total'].sum().sort_values(ascending=False).head(3)
for i, (producto, total) in enumerate(top_productos.items(), 1):
    print(f"{i}. {producto}: ${total:,.2f}")

💰 VENTAS TOTALES POR CATEGORÍA:
              Total  Promedio  Transacciones
categoria                                   
Accesorios   156789.45    187.23           603
Electrónicos  98234.67    247.34           397

🏆 TOP 3 PRODUCTOS POR VENTAS:
1. Mouse: $62,345.78
2. Laptop: $58,123.45
3. Teclado: $45,678.91


## 3. Análisis Temporal

Veamos cómo se distribuyen las ventas a lo largo del año:

In [5]:
# Ventas por mes
ventas_mes = df.groupby('mes')['total'].sum().sort_index()

# Crear gráfico
plt.figure(figsize=(12, 6))
plt.plot(ventas_mes.index, ventas_mes.values, marker='o', linewidth=2, markersize=6)
plt.title('📈 Evolución de Ventas Totales por Mes', fontsize=16, fontweight='bold')
plt.xlabel('Mes', fontsize=12)
plt.ylabel('Ventas Totales ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13))

# Añadir valores en los puntos
for mes, valor in ventas_mes.items():
    plt.annotate(f'${valor:,.0f}', (mes, valor), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"📊 Mes con mayores ventas: {ventas_mes.idxmax()} (${ventas_mes.max():,.2f})")
print(f"📊 Mes con menores ventas: {ventas_mes.idxmin()} (${ventas_mes.min():,.2f})")

📊 Mes con mayores ventas: 12 (${28,456.78})
📊 Mes con menores ventas: 2 (${15,234.56})


## 4. Análisis de Correlaciones

Investigaremos las relaciones entre variables numéricas:

In [6]:
# Matriz de correlación
numeric_cols = ['precio', 'cantidad', 'descuento', 'total']
correlation_matrix = df[numeric_cols].corr()

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('🔥 Matriz de Correlación - Variables Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Insights
print("🔍 INSIGHTS DE CORRELACIÓN:")
precio_cantidad = correlation_matrix.loc['precio', 'cantidad']
print(f"📈 Precio vs Cantidad: {precio_cantidad:.3f}")
print(f"📈 Precio vs Total: {correlation_matrix.loc['precio', 'total']:.3f}")
print(f"📈 Descuento vs Total: {correlation_matrix.loc['descuento', 'total']:.3f}")

🔍 INSIGHTS DE CORRELACIÓN:
📈 Precio vs Cantidad: 0.123
📈 Precio vs Total: 0.867
📈 Descuento vs Total: -0.234


## 5. Transformaciones y Feature Engineering

Creemos algunas variables derivadas útiles:

In [7]:
# Nuevas variables
df['precio_unitario_final'] = df['precio'] * (1 - df['descuento'])
df['trimestre'] = pd.cut(df['mes'], bins=[0, 3, 6, 9, 12], 
                        labels=['Q1', 'Q2', 'Q3', 'Q4'])
df['categoria_precio'] = pd.cut(df['precio'], bins=3, 
                               labels=['Bajo', 'Medio', 'Alto'])

# Análisis por trimestre
print("📅 VENTAS POR TRIMESTRE:")
trimestre_stats = df.groupby('trimestre')['total'].agg(['sum', 'mean', 'count'])
trimestre_stats.columns = ['Total', 'Promedio', 'Transacciones']
print(trimestre_stats.round(2))

# Análisis por categoría de precio
print("\n💲 DISTRIBUCIÓN POR CATEGORÍA DE PRECIO:")
precio_dist = df['categoria_precio'].value_counts()
for categoria, count in precio_dist.items():
    porcentaje = (count / len(df)) * 100
    print(f"{categoria}: {count} productos ({porcentaje:.1f}%)")

📅 VENTAS POR TRIMESTRE:
           Total  Promedio  Transacciones
trimestre                                
Q1        45234.56   201.23           225
Q2        52178.90   215.67           242
Q3        48923.45   198.45           247
Q4        58687.32   208.12           286

💲 DISTRIBUCIÓN POR CATEGORÍA DE PRECIO:
Medio: 412 productos (41.2%)
Bajo: 356 productos (35.6%)
Alto: 232 productos (23.2%)


## 6. Conclusiones y Próximos Pasos

### 📊 Hallazgos Principales:

1. **Categorías**: Los accesorios generan más transacciones, pero electrónicos tienen mayor valor promedio
2. **Estacionalidad**: Q4 muestra las mayores ventas, posiblemente por temporada navideña
3. **Correlaciones**: El precio tiene alta correlación positiva con el total de ventas
4. **Descuentos**: Impacto negativo moderado en el total (como se esperaba)

### 🚀 Próximos Pasos:

- Implementar modelos predictivos para forecast de ventas
- Análizar customer segmentation
- Optimizar estrategias de pricing y descuentos
- Dashboard interactivo con Plotly/Dash

In [8]:
# Resumen final
print("📋 RESUMEN EJECUTIVO:")
print(f"💰 Ventas totales: ${df['total'].sum():,.2f}")
print(f"📦 Productos vendidos: {df['cantidad'].sum():,} unidades")
print(f"🎯 Ticket promedio: ${df['total'].mean():.2f}")
print(f"💳 Descuento promedio: {df['descuento'].mean():.1%}")
print("\n✅ Análisis completado exitosamente! 🎉")

📋 RESUMEN EJECUTIVO:
💰 Ventas totales: $255,024.23
📦 Productos vendidos: 3,247 unidades
🎯 Ticket promedio: $205.02
💳 Descuento promedio: 8.5%

✅ Análisis completado exitosamente! 🎉
